In [10]:
from odyssey.core.bigquery.BigQueryGithubEntry import *
from odyssey.core.bigquery.GithubPython import GithubPython
from odyssey.core.analyzer import InstantiationAnalyzer
from odyssey.core.analyzer import ImportAnalyzer
import matplotlib.pyplot as plt
import numpy as np
from nbconvert import PythonExporter
import pandas as pd
import pickle
from odyssey.core.analyzer import InstantiationAnalyzer
import glob
import ast
import jedi

import csv
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from odyssey.core.bigquery.GithubPython import GithubPython as ghp
%matplotlib notebook 

In [2]:
class FunctionCallVisitor(ast.NodeVisitor):
    def visit_Call(self, node):
        print (ast.dump(node))

In [3]:
functionsList=['RF', 'GridSearchCV','make_pipeline','AdaBoostRegressor']

## Extracting files in repo containing scikit-learn

In [5]:
query = """\
SELECT
    DISTINCT(repo_name)
FROM
    `Odyssey_github_sklearn.content_py_full` 
WHERE
    (REGEXP_CONTAINS(path,"sklearn")) OR
    (REGEXP_CONTAINS(repo_name,"scikit-learn")) OR
    (REGEXP_CONTAINS(repo_name,"sklearn")) OR
    (REGEXP_CONTAINS(path,"scikit-learn"))
"""
gp = GithubPython()



excludedRepos = [repo_name[0] for repo_name in gp.run(query, project="odyssey-193217")]
print(excludedRepos)

In [6]:
def excludeByRepoFull(excludeByRepos):
    if not excludedRepos:
        return ""
    s = '(NOT REGEXP_CONTAINS(repo_name,"%s"))' % excludedRepos[0]
    for repo in excludedRepos[1:]:
        s += ' AND '
        s += '(NOT REGEXP_CONTAINS(repo_name,"%s"))' % repo
    return s

In [4]:
functionListFromFile= pd.read_csv("classNames.csv")
functionListFromFile.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
functionListFromFile = np.array(functionListFromFile)
for file in functionListFromFile:
    print(file[0])

ARDRegression
AdaBoostClassifier
AdaBoostRegressor
AdditiveChi2Sampler
AffinityPropagation
AgglomerativeClustering
BaggingClassifier
BaggingRegressor
BayesianGaussianMixture
BayesianRidge
BernoulliNB
BernoulliRBM
Binarizer
Birch
CCA
CalibratedClassifierCV
CheckingClassifier
ClassifierChain
CountVectorizer
DBSCAN
DPGMM
DecisionTreeClassifier
DecisionTreeRegressor
DictVectorizer
DictionaryLearning
DummyClassifier
DummyRegressor
ElasticNet
ElasticNetCV
EllipticEnvelope
EmpiricalCovariance
ExtraTreeClassifier
ExtraTreeRegressor
ExtraTreesClassifier
ExtraTreesRegressor
FactorAnalysis
FastICA
FeatureAgglomeration
FeatureHasher
FeatureUnion
FunctionTransformer
GMM
GaussianMixture
GaussianNB
GaussianProcess
GaussianProcessClassifier
GaussianProcessRegressor
GaussianRandomProjection
GaussianRandomProjectionHash
GenericUnivariateSelect
GradientBoostingClassifier
GradientBoostingRegressor
GraphLasso
GraphLassoCV
GridSearchCV
GridSearchCV
HashingVectorizer
HuberRegressor
Imputer
IncrementalPCA
Iso

## All data from py_full

In [ ]:
AllContentQuery=  """\
    SELECT repo_name, content
    FROM  `Odyssey_github_sklearn.content_py_full`
    WHERE (REGEXP_CONTAINS(content, "from sklearn+ ") OR REGEXP_CONTAINS(content, "import sklearn+ ")) AND %s 
    GROUP BY 1,2
    """ %excludeByRepoFull(excludedRepos)

AllContent = gp.run(AllContentQuery)

AllContent_df = []

for i in range(len(AllContent)):
    AllContent_df.append((AllContent[i][0],AllContent[i][1]))
    
AllContent_df = pd.DataFrame(AllContent_df, columns= ['repo','content'])
AllContent_df.to_csv('AllContent_df.csv', encoding='utf-8', index=False)

In [6]:
AllContent_df=pd.read_csv('AllContent_df.csv', encoding ='utf-8')
AllContent_py_unique_df = pd.read_csv('AllContent_unique.csv', encoding='utf-8')
AllContent_ipynb_df =pd.read_csv('AllContent_ipynb_df.csv', encoding='utf-8')

## All data from py_unique

In [ ]:
AllContentQuery_unique=  """\
    SELECT repo_name, content
    FROM  `Odyssey_github_sklearn.content_py_unique`
    WHERE (REGEXP_CONTAINS(content, "from sklearn+ ") OR REGEXP_CONTAINS(content, "import sklearn+ ")) AND %s 
    GROUP BY 1,2
    """ %excludeByRepoFull(excludedRepos)

AllContent_unique = gp.run(AllContentQuery_unique)

AllContent_py_unique_df = []

for i in range(len(AllContent_unique)):
    AllContent_py_unique_df.append((AllContent_unique[i][0],AllContent_unique[i][1]))
    
AllContent_py_unique_df = pd.DataFrame(AllContent_py_unique_df, columns= ['repo','content'])
AllContent_py_unique_df.to_csv('AllContent_unique.csv', encoding='utf-8', index=False)

In [ ]:
query_ipynb = """\
SELECT
    DISTINCT(sample_repo_name)
FROM
    `Odyssey_github_sklearn.content_ipynb` 
WHERE
    (REGEXP_CONTAINS(sample_path,"sklearn")) OR
    (REGEXP_CONTAINS(sample_repo_name,"scikit-learn")) OR
    (REGEXP_CONTAINS(sample_repo_name,"sklearn")) OR
    (REGEXP_CONTAINS(sample_path,"scikit-learn"))
"""
gp = GithubPython()



excludedRepos_ipynb = [sample_repo_name[0] for sample_repo_name in gp.run(query_ipynb, project="odyssey-193217")]
print(excludedRepos_ipynb)

In [ ]:
def excludeByRepoFull_ipynb(excludeByRepos):
    if not excludedRepos:
        return ""
    s = '(NOT REGEXP_CONTAINS(sample_repo_name,"%s"))' % excludedRepos[0]
    for repo in excludedRepos[1:]:
        s += ' AND '
        s += '(NOT REGEXP_CONTAINS(sample_repo_name,"%s"))' % repo
    return s

In [ ]:
AllContentQuery_ipynb=  """\
    SELECT sample_repo_name, content
    FROM  `Odyssey_github_sklearn.content_ipynb`
    WHERE (REGEXP_CONTAINS(content, "from sklearn+ ") OR REGEXP_CONTAINS(content, "import sklearn+ ")) AND %s 
    GROUP BY 1,2
    """ %excludeByRepoFull_ipynb(excludedRepos_ipynb)

AllContent_ipynb = gp.run(AllContentQuery_ipynb)

AllContent_ipynb_df = []

for i in range(len(AllContent_ipynb)):
    AllContent_ipynb_df.append((AllContent_ipynb[i][0],AllContent_ipynb[i][1]))
    
AllContent_ipynb_df = pd.DataFrame(AllContent_ipynb_df, columns=['repo','content'])
AllContent_ipynb_df.to_csv('AllContent_ipynb_df.csv', encoding='utf-8', index=False)


## Instantiation analyser for py_unique

#### Getting the hyperparameters for OneHotEncode in scikit-learn using Instantiation Analyser

In [20]:
hyperparameter_py_unique_onehotencoder = dict()

for i in tqdm(range(len(AllContent_py_unique_df))):
    
    
            
    #try:
        for file in ['OneHotEncoder']:
            
            if str(file) not in hyperparameter_py_unique_onehotencoder.keys():
                hyperparameter_py_unique_onehotencoder[str(file)]={}
                hyperparameter_py_unique_onehotencoder[str(file)]["file_count"] = 0
                    
                    
            analyzer= InstantiationAnalyzer(str(file))
            analyzer.parse(AllContent_py_unique_df.content[i])

            if(len(analyzer.d.keys())!=0):
                
                hyperparameter_py_unique_onehotencoder[str(file)]["file_count"] += 1
                
                for key in analyzer.d.keys():
                    
                    if key not in hyperparameter_py_unique_onehotencoder[str(file)].keys():
                        hyperparameter_py_unique_onehotencoder[str(file)][key]={}
                        
                    
                    for subkey, subval in analyzer.d[key].items():
                        
                                             
                        if subkey in hyperparameter_py_unique_onehotencoder[str(file)][key].keys():
                            hyperparameter_py_unique_onehotencoder[str(file)][key][subkey] += subval
                        else:
                            hyperparameter_py_unique_onehotencoder[str(file)][key][subkey] = subval
                        
                        
#     except AttributeError:
#         continue    
    

100%|██████████████████████████████████████████████████████████████████████████████| 9246/9246 [13:09<00:00, 11.70it/s]


In [21]:
hyperparameter_py_unique_onehotencoder

{'OneHotEncoder': {'categorical_features': {'[0]': 1,
   'cat': 2,
   'category_indexes': 1,
   'mask': 1},
  'file_count': 7,
  'handle_unknown': {"'42'": 2, "'error'": 2, "'ignore'": 2},
  'n_values': {'2': 2,
   '4': 2,
   '[3, 2, 2]': 2,
   'len(cwdict)+1': 1,
   'len(lexdict)': 1,
   'n_values': 1,
   'np.int': 2},
  'sparse': {'False': 4}}}

#### Getting the hyperparameters for all functions in scikit-learn using Instantiation Analyser

In [7]:
hyperparameter_py_unique = dict()

for i in tqdm(range(len(AllContent_py_unique_df))):
    
    
            
    try:
        for file in functionListFromFile:
            
            if str(file[0]) not in hyperparameter_py_unique.keys():
                hyperparameter_py_unique[str(file[0])]={}
                    
                    
            analyzer= InstantiationAnalyzer(str(file[0]))
            analyzer.parse(AllContent_py_unique_df.content[i])

            if(len(analyzer.d.keys())!=0):
                
                for key in analyzer.d.keys():
                    
                    if key not in hyperparameter_py_unique[str(file[0])].keys():
                        hyperparameter_py_unique[str(file[0])][key]={}
                    
                    for subkey, subval in analyzer.d[key].items():
                        
                                             
                        if subkey in hyperparameter_py_unique[str(file[0])][key].keys():
                            hyperparameter_py_unique[str(file[0])][key][subkey] += subval
                        else:
                            hyperparameter_py_unique[str(file[0])][key][subkey] = subval
                        
                        
    except AttributeError:
        continue    
    

In [140]:
## Saving as a pickle file

with open('hyperparameter_py_unique.pickle', 'wb') as handle:
    pickle.dump(hyperparameter_py_unique, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
## Reading the pickle file
with open('hyperparameter_py_unique.pickle', 'rb') as handle:
    hyperparameter_py_unique = pickle.load(handle)

In [14]:
hyperparameter_py_unique['GridSearchCV']

{'**': {None: 4},
 'cv': {' args.cv': 2,
  ' self.cv': 4,
  '(folds-1)': 2,
  '10': 100,
  '100': 2,
  '2': 14,
  '20': 14,
  '3': 20,
  '4': 6,
  '5': 140,
  'CV_K_FOLD': 8,
  'GRID_SEARCH_CV': 2,
  'K': 2,
  'None': 4,
  'ShuffleSplit(len(y), n_iter=(self.cvFolds+1), test_size=1/float(self.cvFolds), random_state=0)': 4,
  'ShuffleSplit(len(y), n_iterations=(self.cvFolds+1), test_size=1/float(self.cvFolds), random_state=0)': 4,
  'ShuffleSplit(len(y_train_slice), n_iter=(self.cvFolds+1), test_size=1/float(self.cvFolds), random_state=0)': 4,
  'ShuffleSplit(n=n_samples, train_size=train_size,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tn_iter=250, random_state=1)': 2,
  'ShuffleSplit(n=n_samples, train_size=train_size,\r\n                                            n_iter=250, random_state=1)': 2,
  'ShuffleSplit(n_splits=10, test_size=0.2, random_state=10)': 2,
  'ShuffleSplit(test_size=0.20, n_iter=1, random_state=0, n=len(train_images))': 2,
  'StratifiedKFold(y, n_folds=3)': 2,


### Extracting ipython notebook files with Pipeline usage (with InstantiationAnalyser)

In [ ]:

ipynb_pipeline_files = []

for index in tqdm(range(len(AllContent_ipynb_df))):
    
    try:
    
        py_file = "ipython_files/file" +str(index)+".py"

        with open(py_file, 'r',encoding='utf-8') as file:
            file_content = file.read()

        analyzer= InstantiationAnalyzer("Pipeline")
        analyzer.parse(file_content)

        if(len(analyzer.d.keys())!=0):

            ipynb_pipeline_files.append(index)
            
    except:
        continue

with open('ipynb_pipeline_files.pickle', 'wb') as handle:
    pickle.dump(ipynb_pipeline_files, handle, protocol=pickle.HIGHEST_PROTOCOL)


## Instantiation analyser for ipynb files

### Dropping all json ipynb to py files

In [ ]:
for i in range(len(AllContent_ipynb_df)):
    
    file_name = "ipython_files/file"+str(i)+".ipynb "
    print(file_name)


In [ ]:
# Making a list of invalid ipython files

excepted_files = []
for i in tqdm(range(len(AllContent_ipynb_df))):
    
    imp_file_name= "ipython_files/file"+str(i)+".ipynb"
    exp_file_name= "ipython_files/file"+str(i)+".py"
    
    python_exp = PythonExporter()
    
    try:
  
        data = python_exp.from_file(imp_file_name)

        with open(exp_file_name, 'w',encoding='utf-8') as file:
            file.write(str(data))
            
            
    except:
        excepted_files.append((imp_file_name+"\n"))
        continue
    
              
            

#### Getting the hyperparameters for OneHotEncoder in scikit-learn using Instantiation Analyser

In [15]:
hyperparameter_ipynb_onehotencoder = dict()

for py_file in tqdm(glob.glob('ipython_files/*.py')):
    
    with open(py_file, 'r',encoding='utf-8') as file:
        file_content = file.read()
                
               
   # try:
        for function_name in ['OneHotEncoder']:
            
            if str(function_name) not in hyperparameter_ipynb_onehotencoder.keys():
                hyperparameter_ipynb_onehotencoder[str(function_name)]={}
                hyperparameter_ipynb_onehotencoder[str(function_name)]["file_count"]=0
                    
                    
            analyzer= InstantiationAnalyzer(str(function_name))             
            analyzer.parse(file_content)
            
            if(len(analyzer.d.keys())!=0):
                
                hyperparameter_ipynb_onehotencoder[str(function_name)]["file_count"]+=1
                
                for key in analyzer.d.keys():
                    
                    if key not in hyperparameter_ipynb_onehotencoder[str(function_name)].keys():
                        hyperparameter_ipynb_onehotencoder[str(function_name)][key]={}
                    
                    for subkey, subval in analyzer.d[key].items():
                        
                                             
                        if subkey in hyperparameter_ipynb_onehotencoder[str(function_name)][key].keys():
                            hyperparameter_ipynb_onehotencoder[str(function_name)][key][subkey] += subval
                        else:
                            hyperparameter_ipynb_onehotencoder[str(function_name)][key][subkey] = subval
                        
                        
#     except AttributeError:
#         continue    
    


  1%|█                                                                               | 57/4372 [00:09<12:14,  5.87it/s]
Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\aishg\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\aishg\Anaconda3\lib\site-packages\tqdm\_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\aishg\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████████████████████████████████████████████████████████████████████████| 4372/4372 [06:12<00:00, 11.73it/s]


In [16]:
hyperparameter_ipynb_onehotencoder

{'OneHotEncoder': {'categorical_features': {"'all'": 1,
   '[0, 1, 4, 7, 15, 26, 27]': 6,
   '[0]': 2,
   'categ': 2,
   'categorical_indexes': 1,
   'pd.Series(select_features).isin(categorical_features)': 2},
  'dtype': {'int': 2},
  'file_count': 9,
  'n_values': {"mdl['encoder']['values']": 1},
  'sparse': {'False': 2, 'True': 2}}}

In [17]:
## Saving in pickle file

with open('hyperparameter_ipynb_onehotencoder.pickle', 'wb') as handle:
    pickle.dump(hyperparameter_ipynb_onehotencoder, handle, protocol=pickle.HIGHEST_PROTOCOL)


#### Getting the hyperparameters for all functions in scikit-learn using Instantiation Analyser

In [9]:
hyperparameter_ipynb = dict()

for py_file in tqdm(glob.glob('ipython_files/*.py')):
    
    with open(py_file, 'r',encoding='utf-8') as file:
        file_content = file.read()
                
               
    try:
        for function_name in functionListFromFile:
            
            if str(function_name[0]) not in hyperparameter_ipynb.keys():
                hyperparameter_ipynb[str(function_name[0])]={}
                    
                    
            analyzer= InstantiationAnalyzer(str(function_name[0]))             
            analyzer.parse(file_content)
            
            if(len(analyzer.d.keys())!=0):
                
               # print("key not zero")
                
                for key in analyzer.d.keys():
                    
                    if key not in hyperparameter_ipynb[str(function_name[0])].keys():
                        hyperparameter_ipynb[str(function_name[0])][key]={}
                    
                    for subkey, subval in analyzer.d[key].items():
                        
                                             
                        if subkey in hyperparameter_ipynb[str(function_name[0])][key].keys():
                            hyperparameter_ipynb[str(function_name[0])][key][subkey] += subval
                        else:
                            hyperparameter_ipynb[str(function_name[0])][key][subkey] = subval
                        
                        
    except AttributeError:
        continue    
    

In [9]:
## Saving in pickle file

with open('hyperparameter_ipynb.pickle', 'wb') as handle:
    pickle.dump(hyperparameter_ipynb, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [10]:
## Reading the pickle file
with open('hyperparameter_ipynb.pickle', 'rb') as handle:
    hyperparameter_ipynb = pickle.load(handle)

In [42]:
hyperparameter_ipynb['RandomForestClassifier']['max_depth']

{' int(depth)': 1,
 '10': 18,
 '100': 6,
 '12': 4,
 '14': 1,
 '15': 33,
 '150': 1,
 '16': 3,
 '2': 2,
 '22': 2,
 '25': 3,
 '3': 6,
 '30': 2,
 '4': 4,
 '5': 55,
 '50': 1,
 '6': 10,
 '7': 16,
 '8': 14,
 '9': 1,
 'None': 36,
 'best_m': 1,
 'best_max_depth': 3,
 'depth': 1,
 'g': 2,
 'i': 2,
 'm': 1,
 'max_depth': 3}

# Analysis of the scikit-learn module usage with jedi

In [4]:
# # Module Usage analysis 

module_analysis_py_unique = pd.DataFrame(columns=['Filename_index','Description','Package/Module', 'Usage'])
for i in tqdm(range(0, len(AllContent_py_unique_df))):   

    source = AllContent_py_unique_df.content[i]

   
    for pos in range(len(source.splitlines())):
        
        try:
                
            script = jedi.Script(source,pos)
            d = script.goto_definitions()[0]
            if "sklearn" in str(d.full_name):
                
                temp_df = pd.DataFrame([[i,d.description,d.full_name, source.splitlines()[pos-1]]], columns=['Filename_index','Description','Package/Module', 'Usage'])
                module_analysis_py_unique = module_analysis_py_unique.append(temp_df, ignore_index=True)
            
        except:
            continue

            
cols = list(module_analysis_py_unique.columns)
module_analysis_py_unique['count'] = module_analysis_py_unique.groupby(cols)['Filename_index'].transform('size')
module_analysis_py_unique.drop_duplicates(inplace=True)
module_analysis_py_unique.reset_index(drop=True, inplace=True)

100%|███████████████████████████████████████████████████████████████████████████| 2954/2954 [12:24:17<00:00, 15.12s/it]


In [79]:
with open('module_analysis_py_unique.pickle', 'wb') as handle:
    pickle.dump(module_analysis_py_unique, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [6]:
with open('module_analysis_py_unique.pickle', 'rb') as handle:
    module_analysis_py_unique = pickle.load(handle)
   

## Hyperparameter analysis of pipeline before and after jedi parsing

In [8]:
MA_file_py_unique = module_analysis_py_unique['Filename_index'].unique()
                                           

In [30]:
## PIPELINE ANALYSIS AFTER JEDI


hyperparameter_py_unique_after_jedi_Pipeline = dict()
after_jedi_pipeline_files=[]

for i in tqdm(MA_file_py_unique):
            
    try:       
            
        analyzer= InstantiationAnalyzer("Pipeline")
        analyzer.parse(AllContent_py_unique_df.content[i])

        if(len(analyzer.d.keys())!=0):
            
            after_jedi_pipeline_files.append(i)

            for key in analyzer.d.keys():
                
                if key not in hyperparameter_py_unique_after_jedi_Pipeline.keys():
                    hyperparameter_py_unique_after_jedi_Pipeline[key]={}

                for subkey, subval in analyzer.d[key].items():


                    if subkey in hyperparameter_py_unique_after_jedi_Pipeline[key].keys():
                        hyperparameter_py_unique_after_jedi_Pipeline[key][subkey] += subval
                    else:
                        hyperparameter_py_unique_after_jedi_Pipeline[key][subkey] = subval
                        
                        
    except AttributeError:
        continue    
    

100%|██████████████████████████████████████████████████████████████████████████████| 8860/8860 [10:20<00:00, 14.27it/s]


In [31]:
## saving to pickle file
with open('hyperparameter_py_unique_after_jedi_Pipeline.pickle', 'wb') as handle:
    pickle.dump(hyperparameter_py_unique_after_jedi_Pipeline, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
with open('after_jedi_pipeline_files.pickle', 'wb') as handle:
    pickle.dump(after_jedi_pipeline_files, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
## PIPELINE ANALYSIS AFTER JEDI


hyperparameter_py_unique_before_jedi_Pipeline = dict()
before_jedi_pipeline_files=[]

for i in tqdm(range(len(AllContent_py_unique_df))):
            
    try:
            
        analyzer= InstantiationAnalyzer("Pipeline")
        analyzer.parse(AllContent_py_unique_df.content[i])

        if(len(analyzer.d.keys())!=0):
            
            before_jedi_pipeline_files.append(i)
            

            for key in analyzer.d.keys():

                if key not in hyperparameter_py_unique_before_jedi_Pipeline.keys():
                    hyperparameter_py_unique_before_jedi_Pipeline[key]={}

                for subkey, subval in analyzer.d[key].items():


                    if subkey in hyperparameter_py_unique_before_jedi_Pipeline[key].keys():
                        hyperparameter_py_unique_before_jedi_Pipeline[key][subkey] += subval
                    else:
                        hyperparameter_py_unique_before_jedi_Pipeline[key][subkey] = subval
                        
                        
    except AttributeError:
        continue    
    

In [ ]:
## saving to pickle filea
with open('hyperparameter_py_unique_before_jedi_Pipeline.pickle', 'wb') as handle:
    pickle.dump(hyperparameter_py_unique_before_jedi_Pipeline, handle, protocol=pickle.HIGHEST_PROTOCOL)s

In [ ]:
with open('after_jedi_pipeline_files.pickle', 'rb') as handle:
    after_jedi_pipeline_files = pickle.load(handle)

## Extract files containing make_pipiline using BiqQuery API

In [ ]:
make_pipelineQuery = """\
    SELECT repo_name, line
    FROM (
          SELECT DISTINCT(line), repo_name
          FROM(
          SELECT (SPLIT(content, '\\n')) as lines, repo_name, content
          FROM
          `Odyssey_github_sklearn.content_py_unique`
          ), UNNEST(lines) line 

          WHERE (REGEXP_CONTAINS(line, 'make_pipeline')) AND NOT (REGEXP_CONTAINS(line, 'print')) AND NOT (REGEXP_CONTAINS(line, 'import')) AND (REGEXP_CONTAINS(content, 'from sklearn+  import') OR REGEXP_CONTAINS(content, 'import sklearn+ ')) AND %s
         )
     
        GROUP BY 1,2
    
        """ %( excludeByRepoFull(excludedRepos))

    
    
make_pipeline_result = gp.run(make_pipelineQuery)
